In [10]:
import ezdxf
doc=ezdxf.readfile("test.dxf")

### 布局与块

在DXF文件中，paperspace和modelspace是两个不同的工作空间。

Modelspace是绘图的实际空间，它包含实际的尺寸和坐标值，并且用于创建和编辑实际的几何图形。

Paperspace是用于布局和显示绘图的虚拟空间。在paperspace中，您可以将模型视图放置在页面布局中并添加注释、标题块等信息，以创建最终的绘图输出。

布局是DXF实体（例如LINE或CIRCLE）的容器。最重要的布局是CAD应用程序中标记为“Model”的模型空间，它代表了“世界”工作空间。Paperspace布局代表可绘制的图纸，其中包含绘图的框架和瓷砖块以及VIEWPORT实体，这些实体是缩放和裁剪的“窗口”，可以查看模型空间。

模型空间始终存在且不能被删除。活动paperspace也始终存在于新的DXF文档中，但可以被删除，这种情况下另一个paperspace布局成为新的活动paperspace，但您不能删除最后一个paperspace布局。

获取DXF文档的模型空间：

In [12]:
msp = doc.modelspace()

通过CAD应用程序选项卡中显示的名称获取paperspace布局：

In [14]:
psp = doc.paperspace("布局1")

块只是另一种实体空间，可以通过INSERT实体（也称为块引用）在其他布局和块中多次插入，这是DXF格式的非常强大和重要的概念。

通过块名称获取块布局：

In [ ]:
blk = doc.blocks.get("NAME")

所有这些布局都具有工厂函数(factory function)来为其实体空间创建图形DXF实体。

### 查询DXF实体

如“布局和块”部分所述，所有图形DXF实体都存储在布局中，可以迭代所有这些布局，并支持索引运算符，例如layout[-1]返回最后一个实体。

迭代和索引访问之间的主要区别是，迭代会过滤已销毁的实体，但索引运算符返回已销毁的实体，直到通过`layout.purge()`清除这些实体，有关此主题的更多信息，请参见“删除实体”一节。

有两种高级查询方法：`query()`和`groupby()`。

获取“MyLayer”层的所有线条：

In [15]:
lines = msp.query('LINE[layer=="MyLayer"]')

这将返回一个EntityQuery容器，它还提供相同的`query()`和`groupby()`方法。

在DXF文件中，颜色被编码为数字。预定义的一些常用颜色（例如红色、绿色、蓝色等）有对应的颜色编号，其中1通常表示红色。因此，如果在代码中使用`all_lines_by_color.get(1, [])`，那么它将返回颜色为红色的所有LINE实体，因为红色通常对应颜色编号1。

按DXF属性（例如颜色）分类的所有线：


In [20]:
all_lines_by_color = msp.query("LINE").groupby("color")#这是一个字典(dict)
lines_with_color_1 = all_lines_by_color.get(1, [])#字典的get方法，用于查找key对应的value，第二个参数表示如果查找不到，则返回空列表
lines_with_color_1 #颜色为1的列表

[<class 'ezdxf.entities.line.Line'> LINE(#8B1),
 <class 'ezdxf.entities.line.Line'> LINE(#944),
 <class 'ezdxf.entities.line.Line'> LINE(#945),
 <class 'ezdxf.entities.line.Line'> LINE(#946),
 <class 'ezdxf.entities.line.Line'> LINE(#947),
 <class 'ezdxf.entities.line.Line'> LINE(#108D),
 <class 'ezdxf.entities.line.Line'> LINE(#108E)]

`groupby()`方法返回一个常规的Python字典，其中颜色作为键，实体的常规Python列表作为值（不是EntityQuery容器）。

### 检查DXF实体

每个DXF实体都有一个dxf命名空间属性，它存储命名的DXF属性。一些实体属性和资源只能从dxf命名空间外部的Python属性或方法中访问，例如LWPOLYLINE实体的顶点。有关每个实体的DXF属性的更多信息，可以在ezdxf.entities模块的文档中找到。

获取一些基本的DXF属性：

In [28]:
entity = msp.query("LINE").first#获取查询结果中的第一个实体
layer = entity.dxf.layer # 获取该实体所在的图层名称。默认为“0”
color = entity.dxf.color # 默认为256 = BYLAYER
layer,color

('EPLAN311', 256)

大多数DXF属性都有默认值，如果DXF属性不存在，则将返回该默认值。对于没有默认值的DXF属性，可以检查该属性是否存在：

In [29]:
entity.dxf.hasattr("true_color")

False

或使用get()方法并提供默认值：

In [30]:
entity.dxf.get("true_color", 0)

0

### 创建一个新的DXF文件
创建最新支持的DXF版本的新文档：

In [54]:
doc = ezdxf.new()

为特定的DXF版本（例如DXF R12）创建新的DXF文档：

In [55]:
doc = ezdxf.new("R12")

ezdxf.new()函数可以通过将参数setup设置为True来创建一些标准资源，例如线型和文本样式：

In [56]:
doc = ezdxf.new(setup=True)

### 创建新的DXF实体
用于创建新的图形DXF实体的工厂方法位于BaseLayout类中，这些工厂方法对于所有实体容器都可用：

- Modelspace（模型空间）
- Paperspace（图纸空间）
- BlockLayout（块布局）

使用方法很简单：

In [57]:
msp = doc.modelspace()
msp.add_line((0, 0), (1, 0), dxfattribs={"layer": "MyLayer"})

<class 'ezdxf.entities.line.Line'> LINE(#8A)

一些重要/必需的DXF属性是显式的方法参数，大多数附加的DXF属性可以通过关键字参数dxfattribs作为常规Python dict对象。支持的DXF属性可以在ezdxf.entities模块的文档中找到。

### 创建新的块
DXF文档的块定义由BlocksSection对象管理：

In [58]:
my_block = doc.blocks.new("MyBlock")

这将创建一个名为"MyBlock"的新块，并将其添加到当前文档的块定义部分。使用该方法还可以获取现有块的引用，如果名称已存在，则返回该块的引用，否则创建一个新块并返回该块的引用。

### 创建块引用（Block Reference）
块引用是另一个名为INSERT的DXF实体。可以使用工厂方法add_blockref()创建Insert实体：

In [59]:
msp.add_blockref("MyBlock", (0, 0))

<class 'ezdxf.entities.insert.Insert'> INSERT(#8E)

这将在模型空间中创建一个新的块引用实体，引用名为"MyBlock"的块，并将其插入到(0, 0)处。可以通过调整插入点的位置、缩放因子、旋转角度等来控制块引用的属性。

### 创建新的图层（Layer）
图层不是实体容器，它只是另一个存储在实体中的DXF属性，实体可以从该图层对象继承一些属性。图层对象存储在图层表中，可以作为doc.layers属性访问。
你可以创建自己的图层：

In [60]:
my_layer = doc.layers.add("MyLayer")

图层对象还控制引用此图层的实体的可见性，该图层的开/关状态遗憾地存储为正或负颜色值，使得图层的原始DXF属性无用。要更改图层的颜色，请使用Layer.color属性：

In [61]:
my_layer.color = 1

要更改图层的状态，请使用Layer对象提供的方法，例如 on()、off()、freeze() 或 thaw() 等：

In [62]:
my_layer.off()

### 删除实体
最安全的删除实体的方法是从包含该实体的布局中删除实体：

In [63]:
line = msp.add_line((0, 9), (1, 0))
msp.delete_entity(line)

这将立即从布局中删除该实体并销毁它。对于已销毁的实体，属性is_alive将返回False，并且所有Python属性都会被删除。因此，line.dxf.color将引发AttributeError异常，因为line不再具有dxf属性。

Ezdxf还支持通过调用destroy()方法手动销毁实体：

In [64]:
line.destroy()

手动销毁实体不会立即从实体容器（如Modelspace或EntityQuery）中删除，但迭代此类容器时将自动过滤已销毁的实体，因此for e in msp: ...循环永远不会产生已销毁的实体。索引运算符和len()函数不过滤已删除的实体，为了避免获取已删除的实体，请手动调用容器的purge()方法以删除已删除的实体。

### 保存DXF文件
使用新名称保存DXF文档：

In [65]:
doc.saveas("new_name.dxf")

或使用与加载的文件相同的名称：

In [66]:
doc.save()#覆盖打开的文件

### 练习时间
1.下面是一段获取模型空间中所有线的代码，已经给出部分代码，请你完成剩余的部分
```Py
import ezdxf
#读取dxf文件
doc = ezdxf.readfile("test.dxf")
#加载模型空间
msp = doc.__________()
#定义打印函数
def print_entity(e):
    print("LINE on layer: %s\n" % e.dxf._______)
    print("start point: %s\n" % e.dxf.start)
    print("end point: %s\n" % e.dxf.end)
#迭代所有的线并打印
for e in msp.________('LINE'):
    print_entity(e)
```

答案：

In [37]:
import ezdxf

doc = ezdxf.readfile('test.dxf')

msp = doc.modelspace()

def print_entity(e):
    print("LINE on layer: %s\n" % e.dxf.layer)
    print("start point: %s\n" % e.dxf.start)
    print("end point: %s\n" % e.dxf.end)

for e in msp.query('LINE'):
    print_entity(e)

LINE on layer: EPLAN311

start point: (213.8145698753751, 222.3204968081464, 0.0)

end point: (217.4625698753752, 222.3204968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 222.3204968081464, 0.0)

end point: (211.3825698753752, 222.3204968081464, 0.0)

LINE on layer: EPLAN533

start point: (213.8145698753751, 221.1044968081464, 0.0)

end point: (213.8145698753751, 213.8084968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 212.5924968081464, 0.0)

end point: (217.4625698753752, 212.5924968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 212.5924968081464, 0.0)

end point: (211.3825698753752, 212.5924968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 122.6084968081464, 0.0)

end point: (217.4625698753752, 122.6084968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 122.6084968081464, 0.0)

end point: (211.3825698753752, 122.6084968081464, 0.0)

LINE on layer: EPLAN533

st